In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

#preprocessing
from unidecode import unidecode
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import OneHotEncoder

#train/test split
from sklearn.model_selection import train_test_split

#vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#performance metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#keras packages
from keras import models
from keras import layers
import tensorflow as tf
from keras import callbacks

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/cat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
#Read the data
df = pd.read_json('News_Category_Dataset_v2.json', lines = True)

df = df.drop(['link', 'date'], axis = 1)

df.head()

,category,headline,authors,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,"The ""Dietland"" actress said using the bags is ..."


In [6]:
df['text'] = df['headline'] + df['short_description']
df.head()

,category,headline,authors,short_description,text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...


In [7]:
def preprocessing(dataframe,punctuation=False,tags=False,stemming=False,lemmatizing=False):
    """
    Function that receives a Pandas DataFrame with the texts and applies
        the chosen preprocessing techiniques.
        
    :param dataframe: a Pandas DataFrame in which the first column 
        contains the estracted texts the second column contains the
        respective authors
    :param punctuation: bool determining whether or remove punctuation
        and numbers or not (default: False)
    :param tags: bool determining whether to remove tags or not
        (default: False)
    :param stemming: bool determining whether to perform stemming or not
        (default: False)
    :param lemmatizing: bool determining whether to perform lemmatizing 
        or not (default: False)
             
    :return: Returns a list of strings which correspond to each text after
        preprocessing
    """
    
    processed_corpus = []
    
    stop_words = set(stopwords.words("english"))
    
    # for each text in the Pandas DataFrame
    for i in tqdm(range(len(dataframe))):
        text = dataframe[i]
                
        # remove punctuation
        if punctuation:
            text = re.sub('[^a-zA-Z]', ' ', text)

        # remove tags
        if tags:
            text = BeautifulSoup(text).get_text()
        
        # convert to list from str
        text = text.split()

        # stemming
        if stemming:
            stemmer = SnowballStemmer('english')
            
            # don't stem stop words so that they can still be detected
            text = [stemmer.stem(word) for word in text if not word in stop_words]
            
        if lemmatizing:
            lemmatizer = WordNetLemmatizer()
            
            text = [lemmatizer.lemmatize(word) for word in text if not word in stop_words]
        
        # convert to str from list
        text = " ".join(text)

        # save the preprocessed text on a list
        processed_corpus.append(text)
    return processed_corpus

In [8]:
# preprocess the text and save it in a Pandas Series
cleaned_text = preprocessing(
    df['text'],
    #punctuation=True,
    #tags=True,
    #stemming=True,
    #lemmatizing=True
)
df['clean_text'] = pd.Series(cleaned_text, index = df.index)

In [9]:
df.head()

,category,headline,authors,short_description,text,clean_text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,Julianna Margulies Uses Donald Trump Poop Bags...


In [10]:
data = df['clean_text']
target = df['category']

In [11]:
X_train,X,y_train,y = train_test_split(data,target,test_size = 0.4,shuffle = True,stratify = target,random_state = 0)

In [12]:
X_val,X_test,y_val,y_test = train_test_split(X,y,test_size = 0.5,shuffle = True,stratify = y,random_state = 0)

In [13]:
vectorizer = CountVectorizer(
    #max_df=0.7,
    #strip_accents='unicode',
    lowercase=False,
    #stop_words=stop_words,
    #max_features=50000,
    #ngram_range=(1,3)
)

enc = OneHotEncoder()

In [14]:
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)

In [15]:
y_train = y_train.values.reshape((120511,1))
y_val = y_val.values.reshape((40171,1))

In [16]:
y_train = enc.fit_transform(y_train)
y_val = enc.transform(y_val)

In [45]:
def dnn(nodes = 64, activation = 'relu', optimizer = 'rmsprop', loss = 'categorical_crossentropy'):
    
    np.random.seed(1)
    tf.random.set_seed(2)
    model = models.Sequential()
    model.add(layers.Dense(nodes,activation=activation,input_shape=(X_train.shape[1],)))
    #model.add(layers.Dropout(0.5))
    #model.add(layers.Dense(nodes,kernel_regularizer=regularizers.l2(0.001),activation=activation,input_shape=(shape,)))
    model.add(layers.Dense(nodes,activation=activation))
    #model.add(layers.Dropout(0.5))
    #model.add(layers.Dense(nodes,kernel_regularizer=regularizers.l2(0.001),activation=activation))
    model.add(layers.Dense(41,activation='softmax'))
    model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
    
    return model

In [46]:
model = dnn()

In [47]:
callbacks_list = [
    callbacks.EarlyStopping(
        monitor = 'val_accuracy',
        patience = 1
    ),
    callbacks.ModelCheckpoint(
        filepath = 'dnn.h5',
        monitor = 'val_accuracy',
        save_best_only = True
    )
]

In [48]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 575, callbacks = callbacks_list, validation_data = (X_val, y_val)) 

Train on 120511 samples, validate on 40171 samples
Epoch 1/100
120511/120511 [==============================] - 154s 1ms/step - loss: 2.3747 - accuracy: 0.4232 - val_loss: 1.8163 - val_accuracy: 0.5393
Epoch 2/100
120511/120511 [==============================] - 154s 1ms/step - loss: 1.4333 - accuracy: 0.6218 - val_loss: 1.5969 - val_accuracy: 0.5797
Epoch 3/100
120511/120511 [==============================] - 151s 1ms/step - loss: 1.0534 - accuracy: 0.7180 - val_loss: 1.5494 - val_accuracy: 0.5939
Epoch 4/100
120511/120511 [==============================] - 151s 1ms/step - loss: 0.7898 - accuracy: 0.7886 - val_loss: 1.5889 - val_accuracy: 0.5932
